In [2]:
import pandas as pd
import numpy as np

In [10]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer


### Prepare data

In [222]:
data_short=pd.read_csv('data/data_short.csv')

In [216]:
data=pd.read_csv('data/epi_data.csv')

In [225]:
ls data

 Volume in drive C is Windows
 Volume Serial Number is 620D-D09A

 Directory of C:\Users\vnuretdi\Documents\Berkeley MIDS\Capstone\Capstone-project\data

10/30/2018  02:25 PM    <DIR>          .
10/30/2018  02:25 PM    <DIR>          ..
10/16/2018  12:52 PM               315 berries.csv
10/30/2018  02:24 PM           249,308 compexity.csv
10/30/2018  02:25 PM           249,308 complexity.csv
10/29/2018  11:44 PM       105,578,382 data_short.csv
10/06/2018  10:34 PM       150,768,319 epi_data.csv
10/16/2018  05:35 PM               932 fruits.csv
10/16/2018  08:04 PM            90,052 ingr_list.csv
10/29/2018  10:58 PM           124,167 ingredients.csv
10/29/2018  12:26 AM           397,457 ingredients_old.csv
10/21/2018  06:36 PM           360,521 meal.csv
10/30/2018  02:21 PM        38,823,821 recipes.json
10/09/2018  02:34 PM               793 spices.csv
08/21/2015  03:08 PM         2,844,086 test.json
08/21/2015  03:09 PM        12,415,067 train.json
10/16/2018  01:03 PM             

In [236]:
# add complexity and cuisine
meal=pd.read_csv('data/meal.csv')
complexity=pd.read_csv('data/complexity.csv')
cuisine=pd.read_csv('data/cuisine.csv') 

meal.drop('0', axis=1, inplace=True)
complexity.drop('0', axis=1, inplace=True) 
cuisine.drop('Unnamed: 0', axis=1, inplace=True)

data_short['meal']=meal
data_short['complexity']=complexity
data_short['cuisine']=cuisine

In [243]:
data_short.to_csv('data/data_short.csv', encoding="UTF-8")

### # Upload the dataset (should be on server)

In [381]:
data_short=pd.read_csv('data/data_short.csv')
data_short.drop('Unnamed: 0', axis=1, inplace=True)

In [382]:
# Upload the list of non_key ingredients
spices=list(pd.read_csv('data/spices.csv')['spices'])
spices=[x for x in spices if x in data_short.columns]

In [383]:
# separate ingredients from non-ingredients 
non_ingredients=['meal','title','calories','protein','carbs','fats','sodium','steps', 'cuisine', 'complexity']
non_ingredients.extend(spices)

### Inputs
 
grocery: list <br>

calories_max: int <br>
calories_min: int <br>
protein_min: int <br>
meal_type: breakfast, lunch, dinner, dessert, drink <br>
cuisine: 20 cuisines in the list <br>
complexity: easy, medium, hard <br>
n_additional_ingredients: int <br>


In [309]:
# Should be at home
home=['pepper', 'butter', 'olive oil', 'sugar', 'salt', 'water', 'lemon juice', 'dijon mustard', 'black pepper', 'ketchup']
grocery=['apple', 'milk', 'hazelnut', 'cucumber', 'bread', 'tomato', 
         'cod fillets',  'rice', 'garlic', 'chicken', 'cheese', 'egg']
grocery.extend(home)

### Function

In [376]:


# quick calculation of additional products
def products_to_add(options, i):
    s=options.loc[i][needed]
    return ', '.join(list(s[s==1].index))

def return_recipes(calories_max=800, 
                   calories_min=500,
                   protein_min=25,
                   meal_type='lunch',
                   cuisine='non_specified',
                   complexity='easy',
                   n_additional_ingredients=4,
                   grocery=grocery):
    if meal_type in ['lunch', 'dinner']: meal_type='lunch/dinner'
    
    #filter basic parameters 
    options=data_short[(data_short.meal==meal_type)&
                       (data_short.calories<calories_max)&
                       (data_short.calories>calories_min)&
                       (data_short.protein>protein_min)&
                       (data_short.complexity=='easy')
                      ]
    if cuisine!='non_specified':
        options=options[options.cuisine==cuisine]
        
    #filter based on grocery
    
    # drop columns with unused ingredients
    ingredients=options.drop(non_ingredients, axis=1).columns
    options.drop([x for x in ingredients if sum(options[x])==0], axis=1, inplace=True)
    
    #update ingredients
    ingredients=options.drop(non_ingredients, axis=1).columns
    
    ## products outside the groccery list   
        
    needed=[x for x in ingredients if x not in grocery]

    # Keep only the recipes if the number of additional key ingredients doesnt exceed 3
    sums=options[needed].sum(axis=1)
    ind=[x for x in sums.index if sums.loc[x]<=n_additional_ingredients]
    options=options.loc[ind]
    
    recommendation=options.loc[ind][['title', 'calories', 'protein', 'carbs', 'fats']]

    recommendation['products to add']=pd.Series([products_to_add(options, i) for i in ind], index=ind)

    recommendation['# of products to add']=pd.Series([len([y for y in recommendation.loc[i]['products to add'].split(",")  
                                                       if y not in non_ingredients]) for i in ind], index=ind)
    
    return recommendation

    

    

### Example

In [380]:
return_recipes(calories_max=800, 
                   calories_min=500,
                   protein_min=25,
                   meal_type='lunch',
                   cuisine='italian',
                   complexity='easy',
                   n_additional_ingredients=4,
                   grocery=grocery)


,title,calories,protein,carbs,fats,products to add,# of products to add
1165,Roasted Beef Tenderloin Wrapped in Bacon,659.0,41.0,4.50,53.0,"bacon, beef",2
3875,Steamed Clams with Pasta,715.0,77.0,79.25,10.0,linguine,1
6748,Lamb with Peas and Tomatoes,568.0,29.0,16.25,43.0,"lamb stew meat, dry wine, pea, sprig",4
7639,Roast New York Strip Loin with Garlic-Herb Crust,504.0,40.0,2.75,37.0,"sage leaves, roast",2
10265,"Roast Leg of Lamb with Mint, Garlic, and Lima ...",685.0,55.0,3.75,50.0,"lamb leg, mint sprigs, kosher salt, bean",4
10574,Sea Bass with Lemon Caper Fettuccine,518.0,39.0,45.50,20.0,"fettuccine, oil, salmon, cayenne",4
11232,Scrambled Egg Pasta,595.0,26.0,50.75,32.0,"bacon, parmesan, pasta, onion",4
11423,Potato and Cheese Ravioli with Fresh Tomato Sa...,629.0,27.0,85.25,20.0,"flour, wonton wrappers, pecorino romano, russe...",4
11812,Lemony Chicken Milanese with Arugula Salad,512.0,59.0,17.25,23.0,"baby arugula, bread crumbs, kosher salt",3
12418,Blade Steaks with Lemon Herb Sauce,543.0,32.0,0.25,46.0,"lemon, leaf parsley",2


### Find pictures for the most common types of foods

In [92]:
titles=list(data_short['title'])

In [93]:
titles=[x.replace(',','') for x in titles]

In [94]:
titles[350]

'Steamed Mussels with Sherry Tomatoes and Garlic '

In [95]:
titles=[x.lower().split() for x in titles]

In [96]:
titles=[[y for y in x if nltk.pos_tag([y])[0][1] in ['NN', 'NNS']] for x in titles]

### Extract top unigrams

In [ ]:
from nltk import word_tokenize
from nltk.collocations import BigramCollocationFinder

In [ ]:
from nltk import word_tokenize 
from nltk.util import ngrams

In [201]:
init_unigrams=[y for x in titles for y in x]

In [203]:
unigrams_count=[(x, init_unigrams.count(x)) for x in set(init_unigrams)]

In [204]:
unigrams_count.sort(key = lambda t: t[1])

In [208]:
unigrams_count[::-1][150:500]

[('sage', 105),
 ('citrus', 105),
 ('prosciutto', 105),
 ('kale', 104),
 ('herbs', 104),
 ('sesame', 103),
 ('parsley', 102),
 ('sugar', 102),
 ('baby', 101),
 ('dip', 101),
 ('buttermilk', 100),
 ('peanut', 99),
 ('balsamic', 99),
 ('banana', 98),
 ('dough', 98),
 ('cauliflower', 98),
 ('pastry', 97),
 ('nuts', 97),
 ('thyme', 96),
 ('watercress', 96),
 ('apples', 95),
 ('ribs', 95),
 ('crispy', 95),
 ('cheesecake', 94),
 ('rum', 94),
 ('maple', 93),
 ('de', 92),
 ('sea', 92),
 ('duck', 91),
 ('hazelnut', 91),
 ('chipotle', 91),
 ('rolls', 90),
 ('burgers', 89),
 ('tea', 89),
 ('summer', 89),
 ('polenta', 88),
 ('vinegar', 87),
 ('punch', 86),
 ('risotto', 86),
 ('spice', 85),
 ('mussels', 84),
 ('bass', 84),
 ('frozen', 84),
 ('barbecue', 82),
 ('spring', 82),
 ('blueberry', 80),
 ('noodles', 79),
 ('cocktail', 78),
 ('saffron', 78),
 ('coffee', 77),
 ('linguine', 77),
 ('chowder', 77),
 ('mousse', 77),
 ('pancakes', 77),
 ('artichoke', 76),
 ('milk', 76),
 ('plum', 76),
 ('lobster',

### Extract bigrams

In [73]:
from nltk import word_tokenize
from nltk.collocations import BigramCollocationFinder

In [74]:
from nltk import word_tokenize 
from nltk.util import ngrams

In [97]:
init_bigrams=[list(ngrams(x, 2)) for x in titles]

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: generator 'ngrams' raised StopIteration
  """Entry point for launching an IPython kernel.


In [98]:
bigrams=[y for x in init_bigrams for y in x]

In [99]:
bigrams_count=[(x, bigrams.count(x)) for x in set(bigrams)]

In [100]:
bigrams_count.sort(key = lambda t: t[1])

In [101]:
bigrams_count=bigrams_count[::-1]

In [198]:
[(x,' '.join(x[0])) for x in bigrams_count[300:400]]

[((('basil', 'sauce'), 11), 'basil sauce'),
 ((('salad', 'sandwiches'), 11), 'salad sandwiches'),
 ((('veal', 'scallops'), 11), 'veal scallops'),
 ((('brussels', 'sprout'), 11), 'brussels sprout'),
 ((('tomato', 'garlic'), 11), 'tomato garlic'),
 ((('onion', 'tomato'), 11), 'onion tomato'),
 ((('roast', 'leg'), 11), 'roast leg'),
 ((('ginger', 'garlic'), 11), 'ginger garlic'),
 ((('pork', 'shoulder'), 11), 'pork shoulder'),
 ((('savoy', 'cabbage'), 11), 'savoy cabbage'),
 ((('chicken', 'bell'), 11), 'chicken bell'),
 ((('corn', 'chowder'), 11), 'corn chowder'),
 ((('pepper', 'tomato'), 11), 'pepper tomato'),
 ((('chipotle', 'sauce'), 11), 'chipotle sauce'),
 ((('beef', 'ribs'), 11), 'beef ribs'),
 ((('pepper', 'relish'), 11), 'pepper relish'),
 ((('seafood', 'stew'), 11), 'seafood stew'),
 ((('matzo', 'balls'), 11), 'matzo balls'),
 ((('salad', 'ginger'), 11), 'salad ginger'),
 ((('spicy', 'shrimp'), 11), 'spicy shrimp'),
 ((('tomato', 'relish'), 11), 'tomato relish'),
 ((('chile', 'sa

In [193]:
[titles[i] for i in range(len(titles)) if ('tomato', 'chutney') in init_bigrams[i]]

[['tomato', 'chutney'],
 ['tomato', 'chutney'],
 ['sweet-and-sour', 'tomato', 'chutney'],
 ['roast',
  'beef',
  'basil',
  'spicy',
  'tomato',
  'chutney',
  'lavash',
  'sandwiches'],
 ['spicy', 'tomato', 'chutney'],
 ['tomato', 'chutney'],
 ['tomato', 'chutney'],
 ['spicy', 'tomato', 'chutney'],
 ['bean', 'salad', 'tomato', 'chutney'],
 ['roast',
  'beef',
  'basil',
  'spicy',
  'tomato',
  'chutney',
  'lavash',
  'sandwiches'],
 ['tomato', 'chutney'],
 ['corn', 'bread', 'tomato', 'chutney']]

### Extract trigrams

In [209]:
init_trigrams=[list(ngrams(x, 3)) for x in titles]

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: generator 'ngrams' raised StopIteration
  """Entry point for launching an IPython kernel.


In [210]:
trigrams=[y for x in init_trigrams for y in x]

In [212]:
trigrams_count=[(x, trigrams.count(x)) for x in set(trigrams)]
trigrams_count.sort(key = lambda t: t[1])
trigrams_count=trigrams_count[::-1]

In [213]:
[(x,' '.join(x[0])) for x in trigrams_count[:200]]

[((('sugar', 'snap', 'peas'), 31), 'sugar snap peas'),
 ((('vanilla', 'ice', 'cream'), 28), 'vanilla ice cream'),
 ((('turkey', 'giblet', 'stock'), 20), 'turkey giblet stock'),
 ((('cake', 'cream', 'cheese'), 17), 'cake cream cheese'),
 ((('goat', 'cheese', 'salad'), 16), 'goat cheese salad'),
 ((('spicy', 'tomato', 'sauce'), 15), 'spicy tomato sauce'),
 ((('ice', 'cream', 'cake'), 13), 'ice cream cake'),
 ((('sugar', 'snap', 'pea'), 12), 'sugar snap pea'),
 ((('baby', 'bok', 'choy'), 12), 'baby bok choy'),
 ((('butternut', 'squash', 'soup'), 12), 'butternut squash soup'),
 ((('roast', 'leg', 'lamb'), 11), 'roast leg lamb'),
 ((('lemon', 'ice', 'cream'), 11), 'lemon ice cream'),
 ((('pico', 'de', 'gallo'), 11), 'pico de gallo'),
 ((('roast', 'pork', 'tenderloin'), 10), 'roast pork tenderloin'),
 ((('bittersweet', 'chocolate', 'sauce'), 10), 'bittersweet chocolate sauce'),
 ((('bell', 'pepper', 'sauce'), 10), 'bell pepper sauce'),
 ((('roast', 'pork', 'loin'), 10), 'roast pork loin'),
 